In [1]:
# Import necessary libraries and configuration of database
import pandas as pd
from sqlalchemy import create_engine # For creating a database connection
from neo4j import GraphDatabase 

import const as c # Import database URL host, userid and password

In [2]:
# Create a PostgreSQL database engine
engine = create_engine(
    f'postgresql://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}', 
    connect_args = {'options': '-c search_path=uadbkt,umobility,public', 'keepalives_idle': 120},
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

# 1. Create graph-based data in Neo4j

## Import table "haltestelle"

In [3]:
# Execute SQL query to retrieve data from the 'haltestelle' table in Postgres
with engine.connect() as con:
    sql = "select * from haltestelle"
    haltestellen_df = pd.read_sql_query(sql, con)
    
haltestellen_df

,hid,bez,lat,lng,pos,posp
0,10001,TheodorHeussPlatz,52.509798,13.272977,0101000020E6100000D11E2FA4C38B2A407AC8940F4141...,0101000020EC7A000058D203B2087F514187C6D6C0F933...
1,10145,Alexanderplatz,52.521512,13.411267,0101000020E61000001BD9959691D22A405C3CBCE7C042...,0101000020EC7A000027AF1D782D8851414CD43F416B35...
2,10146,BerlinHauptbahnhof,52.525847,13.368924,0101000020E6100000446B459BE3BC2A4090D959F44E43...,0101000020EC7A00007F1AF7B35C855141C9C84100D635...
3,10147,BrandenburgerTor,52.516511,13.381937,0101000020E6100000C443183F8DC32A401B834E081D42...,0101000020EC7A0000897EF6753E8651414713C781D634...
4,10148,Bundesplatz,52.477366,13.329149,0101000020E6100000AF05BD3786A82A40761C3F541A3D...,0101000020EC7A000086320CB3D2825141698308C58430...
...,...,...,...,...,...,...
168,10319,WilmersdorferStr,52.506415,13.306774,0101000020E6100000B686527B119D2A40179AEB34D240...,0101000020EC7A0000F05A9E064881514199FDECE5A533...
169,10320,Wittenbergplatz,52.502109,13.342561,0101000020E610000026C5C72764AF2A407D03931B4540...,0101000020EC7A0000CD5C88B3A9835141F30DDF433933...
170,10321,Wutzkyallee,52.423152,13.474820,0101000020E610000033F9669B1BF32A40569E40D82936...,0101000020EC7A0000342364B99B8C51411457CAC4D02A...
171,10322,Zitadelle,52.537522,13.217625,0101000020E61000008195438B6C6F2A40B5705985CD44...,0101000020EC7A000031323734507B51410E7A9AC3EC36...


In [4]:
# Create Neo4j nodes for stations with property constraints
cypher_create_stop = 'CREATE (h:Haltestelle {hid: $hid, bez: $bez, lat:$lat, lng:$lng})'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        for r in haltestellen_df.itertuples(index=False):
            session.run(cypher_create_stop, hid=r.hid, bez=r.bez, lat=r.lat, lng=r.lng)

## Import table "abschnitt"

In [5]:
# Execute SQL query to retrieve data from the 'abschnitt' table in Postgres
with engine.connect() as con:
    sql = "select * from abschnitt"
    abschnitt_df = pd.read_sql_query(sql, con)
    
abschnitt_df

,ulid,nr,hid_a,hid_b,haelt
0,1017001,1,10170,10295,+
1,1017001,2,10295,10219,+
2,1017001,3,10219,10243,+
3,1017001,4,10243,10278,+
4,1017001,5,10278,10222,+
...,...,...,...,...,...
565,1026004,8,10223,10179,+
566,1026004,9,10179,10245,+
567,1026004,10,10245,10303,+
568,1026004,11,10303,10220,-


In [6]:
# Create Neo4j nodes for sections with property constraints
cypher_create_stop = 'CREATE (a:Abschnitt {ulid: $ulid, nr: $nr, hid_a:$hid_a, hid_b:$hid_b, haelt:$haelt})'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        for r in abschnitt_df.itertuples(index=False):
            session.run(cypher_create_stop, ulid=r.ulid, nr=r.nr, hid_a=r.hid_a, hid_b=r.hid_b, haelt=r.haelt)

## Import table "linie"

In [7]:
# Execute SQL query to retrieve data from the 'linie' table in Postgres
with engine.connect() as con:
    sql = "select * from linie"
    linie_df = pd.read_sql_query(sql, con)
    
linie_df

,lid,bez
0,1017,U1
1,1018,U2
2,1019,U3
3,1020,U4
4,1021,U5
5,1022,U55
6,1023,U6
7,1024,U7
8,1025,U8
9,1026,U9


In [8]:
# Create Neo4j nodes for lines with property constraints
cypher_create_stop = 'CREATE (l:Linie {lid: $lid, bez: $bez})'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        for r in linie_df.itertuples(index=False):
            session.run(cypher_create_stop, lid=r.lid, bez=r.bez)

## Import table "unterlinie"

In [9]:
# Execute SQL query to retrieve data from the 'unterlinie' table in Postgres
with engine.connect() as con:
    sql = "select * from unterlinie"
    unterlinie_df = pd.read_sql_query(sql, con)
    
unterlinie_df

,ulid,lid
0,1017001,1017
1,1017003,1017
2,1018001,1018
3,1018003,1018
4,1019001,1019
5,1019003,1019
6,1020001,1020
7,1020003,1020
8,1021001,1021
9,1021003,1021


In [10]:
# Create Neo4j nodes for sublines with property constraints
cypher_create_stop = 'CREATE (ul:Unterlinie {ulid: $ulid, lid: $lid})'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        for r in unterlinie_df.itertuples(index=False):
            session.run(cypher_create_stop, ulid=r.ulid, lid=r.lid)

## Import table "segment"

In [11]:
# Execute SQL query to retrieve data from the 'segment' table in Postgres
with engine.connect() as con:
    sql = "select * from segment"
    segment_df = pd.read_sql_query(sql, con)
    
segment_df

,hid_a,hid_b,laenge_in_meter
0,10001,10263,1152
1,10145,10240,483
2,10145,10294,734
3,10145,10317,590
4,10146,10200,694
...,...,...,...
179,10320,10187,429
180,10320,10245,779
181,10321,10251,814
182,10322,10185,759


In [12]:
# Create Neo4j nodes for segments with property constraints
cypher_create_stop = 'CREATE (s:Segment {hid_a: $hid_a, hid_b: $hid_b, laenge_in_meter: $laenge_in_meter})'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        for r in segment_df.itertuples(index=False):
            session.run(cypher_create_stop, hid_a=r.hid_a, hid_b=r.hid_b, laenge_in_meter=r.laenge_in_meter)

# 2. Create relationships between nodes

In [13]:
cypher_create_stop = 'MATCH (a:Abschnitt), (h:Haltestelle) WHERE h.hid = a.hid_a CREATE (a)-[:ProjAbA]->(h)'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            session.run(cypher_create_stop)

In [14]:
cypher_create_stop = 'MATCH (a:Abschnitt), (h:Haltestelle) WHERE h.hid = a.hid_b CREATE (a)-[:ProjAbB]->(h)'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            session.run(cypher_create_stop)

In [15]:
cypher_create_stop = 'MATCH (s:Segment), (h:Haltestelle) WHERE h.hid = s.hid_a CREATE (s)-[:ProjSegA]->(h)'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            session.run(cypher_create_stop)

In [16]:
cypher_create_stop = 'MATCH (s:Segment), (h:Haltestelle) WHERE h.hid = s.hid_b CREATE (s)-[:ProjSegB]->(h)'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            session.run(cypher_create_stop)

In [17]:
cypher_create_stop = 'MATCH (a:Abschnitt), (ul:Unterlinie) WHERE a.ulid = ul.ulid CREATE (a)-[:InUL]->(ul)'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            session.run(cypher_create_stop)

In [18]:
cypher_create_stop = 'MATCH (ul:Unterlinie), (l:Linie) WHERE ul.lid = l.lid CREATE (ul)-[:InL]->(l)'
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            session.run(cypher_create_stop)

In [19]:
cypher_create_stop = """
MATCH (h1:Haltestelle{hid:$hid_a}), (h2:Haltestelle{hid:$hid_b})
CREATE (h1)-[:Verbindet{laenge_in_meter:$laenge_in_meter}]->(h2), 
 (h2)-[:Verbindet{laenge_in_meter:$laenge_in_meter}]->(h1)
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        for r in segment_df.itertuples(index=False):
            session.run(cypher_create_stop, hid_a=r.hid_a, hid_b=r.hid_b, laenge_in_meter=r.laenge_in_meter)

## Task 1: Lines per sections

In [20]:
neo4j = """
match (h1:Haltestelle)<-[:ProjAbA]-(a:Abschnitt), (h2:Haltestelle)<-[:ProjAbB]-(a:Abschnitt), (a:Abschnitt)-[:InUL]->(u:Unterlinie)-[:InL]->(l:Linie)
return h1.bez as Start, h2.bez as Ende, collect(distinct l.bez) as Linien
order by Start
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            result = session.run(neo4j)
            a1_df = pd.DataFrame(result.data())
            
a1_df

,Start,Ende,Linien
0,Adenauerplatz,WilmersdorferStr,[U7]
1,Adenauerplatz,KonstanzerStr,[U7]
2,AfrikanischeStr,KurtSchumacherPlatz,[U6]
3,AfrikanischeStr,Rehberge,[U6]
4,Alexanderplatz,RosaLuxemburgPlatz,[U2]
...,...,...,...
363,ZoologischerGarten,ErnstReuterPlatz,[U2]
364,ZoologischerGarten,Hansaplatz,[U9]
365,ZoologischerGarten,Kurfürstendamm,[U9]
366,ZwickauerDamm,Wutzkyallee,[U7]


## Task 2:  Subline routes

In [21]:
# A plus or minus sign is to be used to indicate whether the subline stops at the respective station or not. Assume that there is always a stop at the first and last stop of a subline.
q = """MATCH (ha:Haltestelle), (hb:Haltestelle), (a:Abschnitt),(u:Unterlinie),(l:Linie)
WHERE ha.hid = a.hid_a AND hb.hid = a.hid_b AND a.ulid = u.ulid AND u.lid = l.lid
WITH l.bez as Linie, u.ulid as Unterlinie, collect(a.haelt) as h, collect(ha.bez) as has, collect(hb.bez) as hbs
UNWIND has+hbs AS v
WITH Linie, Unterlinie, h, collect(distinct v) as v
WITH Linie, Unterlinie, reverse(reverse(h)+['+']) as h, v
RETURN Linie, Unterlinie, [x in apoc.coll.zip(h,v)|x[0]+x[1]] as Verlauf ORDER BY Linie, Unterlinie
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(q)
        a2_df = pd.DataFrame(result.data())
a2_df

,Linie,Unterlinie,Verlauf
0,U1,1017001,"[+WarschauerStr, +SchlesischesTor, +GörlitzerB..."
1,U1,1017002,"[+UhlandStr, +Kurfürstendamm, +Wittenbergplatz..."
2,U1,1017003,"[+KottbusserTor, -PrinzenStr, +HalleschesTor, ..."
3,U1,1017004,"[+Kurfürstendamm, +Wittenbergplatz, +Nollendor..."
4,U2,1018001,"[+Pankow, +VinetaStr, +SchönhauserAllee, +Eber..."
5,U2,1018002,"[+Ruhleben, +OlympiaStadion, +NeuWestend, +The..."
6,U2,1018003,"[+Alexanderplatz, -KlosterStr, -MärkischesMuse..."
7,U2,1018004,"[+BismarckStr, +DeutscheOper, +ErnstReuterPlat..."
8,U3,1019001,"[+WarschauerStr, +SchlesischesTor, +GörlitzerB..."
9,U3,1019002,"[+KrummeLanke, +OnkelTomsHütte, +OskarHeleneHe..."


# 3. Create projected graph for executing algorithms

In [23]:
neo4j = """
CALL gds.graph.project.cypher('g',
'MATCH (h:Haltestelle) RETURN id(h) AS id',
'MATCH (h1:Haltestelle)<-[sega:ProjSegA]-(s:Segment)-[segb:ProjSegB]->(h2:Haltestelle) 
 RETURN id(h1) AS source, id(h2) AS target, s.laenge_in_meter AS l
 UNION
 MATCH (h1:Haltestelle)<-[segb:ProjSegB]-(s:Segment)-[sega:ProjSegA]->(h2:Haltestelle) 
 RETURN id(h1) AS source, id(h2) AS target, s.laenge_in_meter AS l')
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            result = session.run(neo4j)
            a3_df = pd.DataFrame(result.data())
            
a3_df

,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
0,MATCH (h:Haltestelle) RETURN id(h) AS id,MATCH (h1:Haltestelle)<-[sega:ProjSegA]-(s:Seg...,g,173,368,13


## Task 4: Compute strongly connected components of each station

In [24]:
# Find maximal sets of connected station nodes in a directed graph
neo4j = """
CALL gds.alpha.scc.stream('g')
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).bez AS Haltestelle, componentId AS Komponente
ORDER BY Komponente DESC
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            result = session.run(neo4j)
            a4_df = pd.DataFrame(result.data())
            
a4_df

,Haltestelle,Komponente
0,BerlinHauptbahnhof,2
1,BrandenburgerTor,2
2,Bundestag,2
3,TheodorHeussPlatz,0
4,Alexanderplatz,0
...,...,...
168,WilmersdorferStr,0
169,Wittenbergplatz,0
170,Wutzkyallee,0
171,Zitadelle,0


## Task 5: Shortest paths between two stations

In [25]:
# Find shortest paths, for example, between Tierpark and KochStr
neo4j = """
MATCH (h1:Haltestelle {bez: 'Tierpark'}), (h2:Haltestelle {bez: 'KochStr'})
CALL gds.shortestPath.dijkstra.stream('g', {
    sourceNode: h1,
    targetNode: h2,
    relationshipWeightProperty: 'l'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).bez AS HerkunftHaltestelle,
    gds.util.asNode(targetNode).bez AS ZielHaltestelle,
    totalCost as GesamtLaenge,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).bez] AS Verlauf,
    costs as KumulierteLanenge,
    nodes(path) as Pfade
ORDER BY index
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            result = session.run(neo4j)
            a5_df = pd.DataFrame(result.data())
            
a5_df

,index,HerkunftHaltestelle,ZielHaltestelle,GesamtLaenge,Verlauf,KumulierteLanenge,Pfade
0,0,Tierpark,KochStr,11437.0,"[Tierpark, Friedrichsfelde, Lichtenberg, Magda...","[0.0, 1143.0, 2338.0, 2956.0, 3928.0, 4614.0, ...","[{'hid': 10308, 'lng': 13.523626, 'bez': 'Tier..."


## Task 6: Compute clossness centrality of each station

In [26]:
# Calculate closeness centrality in the network in terms of the length of the segments using Dijkstra single-source shortest path
# For each stop, calculate the sum of the lengths of the shortest paths to all other stops
neo4j="""
MATCH (source:Haltestelle {bez:$bez})
CALL gds.allShortestPaths.dijkstra.stream('g', {
    sourceNode: source,
    relationshipWeightProperty: 'l'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs
WITH
    index,
    gds.util.asNode(sourceNode).bez AS HerkunftHaltestelle,
    gds.util.asNode(targetNode).bez AS ZielHaltestelle,
    totalCost as GesamtLaenge,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).bez] AS nodeNames,
    costs as KumulierteLaenge
return HerkunftHaltestelle, sum(GesamtLaenge) as Summe_der_Länge
"""

a6_df = pd.DataFrame()

with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        for r in haltestellen_df.itertuples(index=False):
            result=session.run(neo4j, bez=r.bez)
            df6 = pd.DataFrame(result.data())
            a6_df = pd.concat([a6_df, df6], axis=0)

a6_df = a6_df.sort_values(by=['Summe_der_Länge'], ascending=True)
a6_df

,HerkunftHaltestelle,Summe_der_Länge
0,Bundestag,1424.0
0,BerlinHauptbahnhof,2118.0
0,BrandenburgerTor,2154.0
0,Gleisdreieck,1216408.0
0,HalleschesTor,1220905.0
...,...,...
0,Kienberg,3514858.0
0,CottbusserPlatz,3635710.0
0,Hellersdorf,3741162.0
0,LouisLewinStr,3908656.0


In [27]:
# Calculate standard closeness centrality for stations nodes (closeness centrality score)
neo4j = """
CALL gds.beta.closeness.stream('g')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).bez AS Haltestelle, (1/score) AS Score
RETURN Haltestelle, Score
ORDER BY Score ASC
"""

with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            result = session.run(neo4j)
            a6_sd_df = pd.DataFrame(result.data())
            
a6_sd_df

,Haltestelle,Score
0,Bundestag,1.000000
1,BerlinHauptbahnhof,1.500000
2,BrandenburgerTor,1.500000
3,HalleschesTor,9.727811
4,Möckernbrücke,9.769231
...,...,...
168,Kienberg,24.313609
169,CottbusserPlatz,25.272189
170,Hellersdorf,26.242604
171,LouisLewinStr,27.224852


## Task 7: Compute betweeness centrality of each station

In [28]:
# Calculate betweeness centrality in the network in terms of the length of the segments using Dijkstra single-source shortest path
# For each stop, calculate the number of shortest paths passing through it
neo4j = """
MATCH (h:Haltestelle {bez: $bez})
CALL gds.allShortestPaths.dijkstra.stream('g', {
    sourceNode: h
})
YIELD index, sourceNode, targetNode, nodeIds
return
    [nodeId IN nodeIds | gds.util.asNode(nodeId).bez] AS nodeNames

"""

a7_df = pd.DataFrame()
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        for r in haltestellen_df.itertuples(index=False):
            result=session.run(neo4j, bez=r.bez)
            df7 = pd.DataFrame(result.data())
            a7_df = pd.concat([a7_df, df7], axis=0)

a7_df

,nodeNames
0,[TheodorHeussPlatz]
1,"[TheodorHeussPlatz, Kaiserdamm]"
2,"[TheodorHeussPlatz, NeuWestend]"
3,"[TheodorHeussPlatz, Kaiserdamm, SophieCharlott..."
4,"[TheodorHeussPlatz, NeuWestend, OlympiaStadion]"
...,...
165,"[ZwickauerDamm, Wutzkyallee, Lipschitzallee, J..."
166,"[ZwickauerDamm, Wutzkyallee, Lipschitzallee, J..."
167,"[ZwickauerDamm, Wutzkyallee, Lipschitzallee, J..."
168,"[ZwickauerDamm, Wutzkyallee, Lipschitzallee, J..."


In [29]:
# Get list of name of nodes 
list_nodes = a7_df['nodeNames']
list_nodes

0                                    [TheodorHeussPlatz]
1                        [TheodorHeussPlatz, Kaiserdamm]
2                        [TheodorHeussPlatz, NeuWestend]
3      [TheodorHeussPlatz, Kaiserdamm, SophieCharlott...
4        [TheodorHeussPlatz, NeuWestend, OlympiaStadion]
                             ...                        
165    [ZwickauerDamm, Wutzkyallee, Lipschitzallee, J...
166    [ZwickauerDamm, Wutzkyallee, Lipschitzallee, J...
167    [ZwickauerDamm, Wutzkyallee, Lipschitzallee, J...
168    [ZwickauerDamm, Wutzkyallee, Lipschitzallee, J...
169    [ZwickauerDamm, Wutzkyallee, Lipschitzallee, J...
Name: nodeNames, Length: 28909, dtype: object

In [31]:
# Remove the first element (sourceNode) and last element (targetNode) of the list if the list is not empty, because the shortes path does not include source node and target node
for n in list_nodes:
    if len(n) != 0:
        n.remove(n[0])
for n in list_nodes:
    if len(n) != 0:
        i = len(n)-1
        n.remove(n[i])
list_nodes

0                                                     []
1                                                     []
2                                                     []
3                                           [Kaiserdamm]
4                                           [NeuWestend]
                             ...                        
165    [Wutzkyallee, Lipschitzallee, JohannisthalerCh...
166    [Wutzkyallee, Lipschitzallee, JohannisthalerCh...
167    [Wutzkyallee, Lipschitzallee, JohannisthalerCh...
168    [Wutzkyallee, Lipschitzallee, JohannisthalerCh...
169    [Wutzkyallee, Lipschitzallee, JohannisthalerCh...
Name: nodeNames, Length: 28909, dtype: object

In [32]:
# Create a new dataframe including list of stations and later storing the number of paths
paths_df = pd.DataFrame(columns=['Stations','Number of paths'])
paths_df['Stations'] = haltestellen_df['bez']
paths_df['Number of paths'] = paths_df['Number of paths'].fillna(0)
paths_df

,Stations,Number of paths
0,TheodorHeussPlatz,0
1,Alexanderplatz,0
2,BerlinHauptbahnhof,0
3,BrandenburgerTor,0
4,Bundesplatz,0
...,...,...
168,WilmersdorferStr,0
169,Wittenbergplatz,0
170,Wutzkyallee,0
171,Zitadelle,0


In [33]:
# Count shortest paths going through each station
for nodes in list_nodes:
    for node in nodes:
        index = paths_df.index[paths_df['Stations']== node].tolist()[0]
        paths_df.at[index,'Number of paths'] +=1

paths_df=paths_df.sort_values(by=['Number of paths'], ascending=False)
paths_df

,Stations,Number of paths
109,Möckernbrücke,9247
1,Alexanderplatz,9041
28,ZoologischerGarten,8213
92,KottbusserTor,8084
71,HalleschesTor,8058
...,...,...
80,Hönow,0
19,RathausSteglitz,0
18,RathausSpandau,0
138,Rudow,0


In [34]:
# Calculate standard betweeness centrality (betweeness centrality score)
neo4j = """
CALL gds.betweenness.stream('g')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).bez AS name, score
ORDER BY score DESC
"""

with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            result = session.run(neo4j)
            a7_sd_df = pd.DataFrame(result.data())
            
a7_sd_df

,name,score
0,Möckernbrücke,9396.100000
1,Alexanderplatz,9032.100000
2,ZoologischerGarten,8263.833333
3,HalleschesTor,8154.100000
4,KottbusserTor,8023.866667
...,...,...
168,Hönow,0.000000
169,KrummeLanke,0.000000
170,Rudow,0.000000
171,Ruhleben,0.000000


## Task 8: Find path between two stations (transit station if necessary) with Python

In [35]:
# Query routes of each line
neo4j = """
match (h:Haltestelle), (a:Abschnitt), (u:Unterlinie), (l:Linie)
where h.hid=a.hid_a and a.ulid=u.ulid and u.lid=l.lid
with l.bez as Linie, collect(distinct h.bez) as Verlauf
return Linie, Verlauf
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
            result = session.run(neo4j)
            routes_df = pd.DataFrame(result.data())
            
routes_df

,Linie,Verlauf
0,U1,"[WarschauerStr, SchlesischesTor, GörlitzerBahn..."
1,U2,"[Pankow, VinetaStr, SchönhauserAllee, Eberswal..."
2,U3,"[WarschauerStr, SchlesischesTor, GörlitzerBahn..."
3,U4,"[InnsbruckerPlatz, RathausSchöneberg, Bayerisc..."
4,U5,"[Alexanderplatz, SchillingStr, StrausbergerPla..."
5,U55,"[BerlinHauptbahnhof, Bundestag, BrandenburgerTor]"
6,U6,"[AltMariendorf, Westphalweg, UllsteinStr, Kais..."
7,U7,"[Rudow, ZwickauerDamm, Wutzkyallee, Lipschitza..."
8,U8,"[HermannStr, LeineStr, BoddinStr, Hermannplatz..."
9,U9,"[RathausSteglitz, SchloßStr, WaltherSchreiberP..."


In [36]:
routes = {}
routes = dict(zip(routes_df['Linie'], routes_df['Verlauf']))
routes

{'U1': ['WarschauerStr',
  'SchlesischesTor',
  'GörlitzerBahnhof',
  'KottbusserTor',
  'PrinzenStr',
  'HalleschesTor',
  'Möckernbrücke',
  'Gleisdreieck',
  'KurfürstenStr',
  'Nollendorfplatz',
  'Wittenbergplatz',
  'Kurfürstendamm',
  'UhlandStr'],
 'U2': ['Pankow',
  'VinetaStr',
  'SchönhauserAllee',
  'EberswalderStr',
  'Senefelderplatz',
  'RosaLuxemburgPlatz',
  'Alexanderplatz',
  'KlosterStr',
  'MärkischesMuseum',
  'Spittelmarkt',
  'Hausvogteiplatz',
  'Stadtmitte',
  'MohrenStr',
  'PotsdamerPlatz',
  'MendelssohnBartholdyPark',
  'Gleisdreieck',
  'BülowStr',
  'Nollendorfplatz',
  'Wittenbergplatz',
  'ZoologischerGarten',
  'ErnstReuterPlatz',
  'DeutscheOper',
  'BismarckStr',
  'SophieCharlottePlatz',
  'Kaiserdamm',
  'TheodorHeussPlatz',
  'NeuWestend',
  'OlympiaStadion',
  'Ruhleben'],
 'U3': ['WarschauerStr',
  'SchlesischesTor',
  'GörlitzerBahnhof',
  'KottbusserTor',
  'PrinzenStr',
  'HalleschesTor',
  'Möckernbrücke',
  'Gleisdreieck',
  'KurfürstenStr

In [37]:
class TrainOrder:
    def __init__(self, train, order, usedTrains):
        self.train = train
        self.order = order
        self.usedTrains = usedTrains

# Get the trains go through each stop
def buildStopToTrainsMap(routes):
    stopToTrains = {}
    for train in routes.keys():
        for stop in routes[train]:
            if stop in stopToTrains:
                stopToTrains[stop].append(train)
            else:
                stopToTrains[stop] = [train]
    return stopToTrains

# Get the trains connect to train
def buildTrainToTrainsMap(stopToTrainsMap, routes):
    trainToConnectingTrainsMap = {}
    for train in routes.keys():
        connectingTrainsSet = []
        for stop in routes[train]:
            connectingTrainsSet.extend(stopToTrainsMap[stop])
        trainToConnectingTrainsMap[train] = list(set(connectingTrainsSet))
    return trainToConnectingTrainsMap

# Get the trains connect to train
def findUsedTrainsUsingBFS(startTrains, endTrains, trainToTrainsMap):
    if startTrains == None or endTrains == None:
        return []
    visitedTrainsSet = []  # List of visited train
    trainOrderQueue = []  # List of train order

    for startTrain in startTrains:
        trainOrderQueue.append(TrainOrder(startTrain, 1, [startTrain]))

    while trainOrderQueue:
        trainOrder = trainOrderQueue.pop(0)
        visitedTrainsSet.append(trainOrder.train)

        if trainOrder.train in endTrains:
            return trainOrder.usedTrains

        connectingTrains = trainToTrainsMap[trainOrder.train]
        for connectingTrain in connectingTrains:
            if not connectingTrain in visitedTrainsSet:
                usedTrains = trainOrder.usedTrains.copy() + [connectingTrain]
                trainOrderQueue.append(TrainOrder(
                    connectingTrain, trainOrder.order + 1, usedTrains))
    return []


def main():
    start = "UhlandStr"
    end = "Hönow"
    stopToTrainsMap = buildStopToTrainsMap(routes)
    trainToTrainsMap = buildTrainToTrainsMap(stopToTrainsMap, routes)
    startTrains = stopToTrainsMap[start]
    endTrains = stopToTrainsMap[end]
    visitedTrains = findUsedTrainsUsingBFS(
        startTrains, endTrains, trainToTrainsMap)
    visitedStop = []

    for key, train in enumerate(visitedTrains):
        if key + 1 < len(visitedTrains):
            stop = [value for value in routes[visitedTrains[key]]
                    if value in routes[visitedTrains[key + 1]]]
            visitedStop.append(stop[0])
    print("Visited trains: ", visitedTrains)
    print("Visited stops/ Stops to transit: ", visitedStop)

In [38]:
main()

Visited trains:  ['U1', 'U2', 'U5']
Visited stops/ Stops to transit:  ['Gleisdreieck', 'Alexanderplatz']
